In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
#from implementations import *
#from split_data import * 
%load_ext autoreload
%autoreload 2


In [ ]:
pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 30.6 MB/s 
     |████████████████████████████████| 5.5 MB 61.6 MB/s 
     |████████████████████████████████| 1.3 MB 54.3 MB/s 
     |████████████████████████████████| 441 kB 66.1 MB/s 
     |████████████████████████████████| 1.9 MB 55.5 MB/s 


In [3]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

ModuleNotFoundError: ignored

In [ ]:
# Main external library : Natural Language Toolkit (nltk)
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()

nltk.download('punkt')
from nltk.corpus import stopwords

### Load tweets

In [ ]:
DIRECTORY = "data/"

# Note: it seems that the data is already to lower case, so no need to apply lower() to the text
pos_data = pd.read_fwf("/content/train_neg_full.txt", header=None, names=["text"]).drop_duplicates().apply(lambda x: x.str.lower())
pos_data["labels"] = 1
neg_data = pd.read_fwf('/content/train_pos_full.txt', header=None, names=["text"]).drop_duplicates().apply(lambda x: x.str.lower())
neg_data["labels"] = 0

### Cleaning Data

In [ ]:
from collections import Counter
import re
import string
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
pd.set_option('display.max_colwidth',100)

In [ ]:
# A word that is so common that there is no need to use it in a search
ENGLISH_STOP_WORDS = nltk.corpus.stopwords.words('english')

# Adding few extra stop word
ENGLISH_STOP_WORDS = ENGLISH_STOP_WORDS + ['im', 'dont','dunno', 'cant', ' 2 ', "'s", ' u ', ' x ']

In [ ]:
#Concatenate the two training sets of positive and negative tweets
train_data = pd.concat([pos_data, neg_data], ignore_index=True)

In [ ]:
train_data = train_data.sample(frac = 1)


In [ ]:
train_data

In [ ]:
# Calculate the most common words used in the set of all tweets
def get_most_common_words(txt,limit):
    return Counter(txt.split()).most_common()[:limit]

# Remove from tweets the punctuation and stop words (= a word that is so common that there is no need to use it in a search.)
def clean_tweet(tweet):
    tweet = "".join([w for w in tweet if w not in string.punctuation])
    tokens = re.split('\W+', tweet)
    tweet = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    return tweet

# Change any word belonging to the same word-family into a common word (changing/changes/changed.. ==> change)
def lemmatization(token_tweet):
    tweet = [wn.lemmatize(word) for word in token_tweet]
    return tweet

# Concatenate the tokennized tweet into a all text like at the beginning
def concatenate(lst):
    concatenate_tweet = ''
    for elem in lst:
        concatenate_tweet = concatenate_tweet + ' ' + elem
    return concatenate_tweet
#we should remove also the numeber appeared in the tweet since doesn't contains any information 

In [ ]:
# Apply the clean_tweet transformation
train_data['text'] = train_data['text'].apply(lambda x : clean_tweet(x)).apply(lambda x : lemmatization(x))

In [ ]:
train_data

### Creation of the model

1. Initialize a task-specific model

In [ ]:
model = ClassificationModel("roberta", "roberta-base", use_cuda=False)

2. Train the model with train_model()

In [ ]:
model.train_model(train_data.sample(frac=0.005), output_dir="outputs/roberta", args={"overwrite_output_dir": True, "num_train_epochs": 1})

3. Evaluate the model

In [ ]:
# sample an other part of the train full dataset and measure the accuracy
# with eval_model()
result, model_outputs, wrong_predictions = model.eval_model(train_data.sample(frac=0.005))

In [ ]:
result, model_outputs

4. Make predictions on (unlabelled) data

In [ ]:
# create the submission file on the test dataset
#  with predict()